In [1]:
import os
import json
import pandas as pd

In [4]:
#from google.colab import files # solo para google collab
from google.cloud import storage
import pandas as pd
print("storage==",storage.__version__)

# entorno local
path_api_key = '/Users/manuel/gdrive/prgrmmng/gcloud_api_key/extreme-unison-399121-cadd77c555ca.json'

# extraigo las credenciales para el acceso
service_account_info = json.load(open(path_api_key))

# autorizo el acceso a cloud storage
client_storage = storage.Client.from_service_account_info(service_account_info)

# para entorno Google Colab o maquina local:
bucket = client_storage.get_bucket('1_transform')
#data_blob = bucket.blob('data_gmaps_reviews_norm.parquet')
data_blob = bucket.blob('gmaps_metadata_filtrado.parquet')
data_gmaps_metadata = pd.read_parquet(storage.fileio.BlobReader(data_blob))

storage== 2.6.0


In [5]:
data_gmaps_metadata = data_gmaps_metadata.drop(
    columns=[
        "relative_results"
        #,"hours"
        ,"state"]
    ).rename(columns={"estado_categoria":"hours-state_cat"}).copy()
data_gmaps_metadata.info()
data_gmaps_metadata.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 441868 entries, 11 to 3024974
Data columns (total 14 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   name             441868 non-null  object 
 1   address          439290 non-null  object 
 2   gmap_id          441868 non-null  object 
 3   description      197047 non-null  object 
 4   latitude         441868 non-null  float64
 5   longitude        441868 non-null  float64
 6   category         441868 non-null  object 
 7   avg_rating       441868 non-null  float64
 8   num_of_reviews   441868 non-null  int64  
 9   price            244781 non-null  object 
 10  hours            401547 non-null  object 
 11  MISC             436358 non-null  object 
 12  url              441868 non-null  object 
 13  hours-state_cat  441868 non-null  object 
dtypes: float64(3), int64(1), object(10)
memory usage: 50.6+ MB


,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,url,hours-state_cat
11,The Jambalaya Shoppe,"The Jambalaya Shoppe, 2108 W Thomas St A, Hamm...",0x862722f48f2b47ef:0x2d579bd17f79bf9b,Casual eatery known for its jambalaya & other ...,30.501556,-90.484082,cajun restaurant,4.0,248,$,"[[Wednesday, 10:30AM–8PM], [Thursday, 10:30AM–...",{'Accessibility': ['Wheelchair accessible entr...,https://www.google.com/maps/place//data=!4m2!3...,Open
11,The Jambalaya Shoppe,"The Jambalaya Shoppe, 2108 W Thomas St A, Hamm...",0x862722f48f2b47ef:0x2d579bd17f79bf9b,Casual eatery known for its jambalaya & other ...,30.501556,-90.484082,restaurant,4.0,248,$,"[[Wednesday, 10:30AM–8PM], [Thursday, 10:30AM–...",{'Accessibility': ['Wheelchair accessible entr...,https://www.google.com/maps/place//data=!4m2!3...,Open


In [6]:
# justificacion:
#data_gmaps_metadata.drop_duplicates(inplace=True) # TypeError: unhashable type: 'list'

In [7]:
column_nonlist = [
    "gmap_id"
    ,"name"
    ,"address"
    ,"description"
    ,"latitude"
    ,"longitude"
    ,"category"
    ,"avg_rating"
    ,"num_of_reviews"
    ,"price"
    #,"MISC"
    #,"hours"
    ,"hours-state_cat"
    ]
column_list_misc = [
    "gmap_id"
    ,"MISC"
    #,"hours"
    ]
column_list_hours = [
    "gmap_id"
    #,"MISC"
    ,"hours"
    ]

## Remover filas duplicadas: Columnas sin listas

In [8]:
# nos quedamos con las filas unicas de las columnas sin listas
data_gmaps_metadata_nonlist = data_gmaps_metadata[
    column_nonlist].drop_duplicates().sort_values(by=['gmap_id']).copy()
data_gmaps_metadata_nonlist.head(2)

,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat
2381893,0x0:0x2ab217fc59211262,Sidelines Grille,"Sidelines Grille, 3405 Busbee Dr NW, Kennesaw,...",None,34.034708,-84.575423,restaurant,2.3,3,None,Specific Hours
1321055,0x14129c7a5c80b46d:0x33149e690acebdf,Cindy Hutson Cuisine,"Cindy Hutson Cuisine, 278 Miracle Mile, Coral ...",None,25.749203,-80.260267,caribbean restaurant,5.0,4,None,Specific Hours


In [9]:
# Contamos el numero de caracteres en 'category'
data_gmaps_metadata_nonlist["category_length"] = data_gmaps_metadata_nonlist["category"].str.len()
# Nos quedamos con los valores en 'category_length'que contienen mayor info; 
# keep='last' corresponde a la fila donde 'category_length' es mas alto
data_gmaps_metadata_nonlist_dupslen = data_gmaps_metadata_nonlist[
    data_gmaps_metadata_nonlist.duplicated(
        subset=['gmap_id'],keep='last')
    ].sort_values(by=['gmap_id','category_length']).drop(columns=['category_length'])
data_gmaps_metadata_nonlist_dupslen.head(2)

,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat
1321055,0x14129c7a5c80b46d:0x33149e690acebdf,Cindy Hutson Cuisine,"Cindy Hutson Cuisine, 278 Miracle Mile, Coral ...",None,25.749203,-80.260267,caribbean restaurant,5.0,4,None,Specific Hours
768183,0x14589e44e1543481:0x488689a13c898c2d,Horizons,"Horizons, 5472 1st Coast Hwy, Fernandina Beach...",None,30.573791,-81.452990,american restaurant,4.3,58,$$,Permanently closed


In [10]:
# df sin las filas con datos duplicados
data_gmaps_metadata_nonlist_unique = data_gmaps_metadata_nonlist[
    ~data_gmaps_metadata_nonlist.duplicated(
        subset=['gmap_id'],keep=False)
    ].sort_values(by=['gmap_id']).drop(columns=['category_length'])
data_gmaps_metadata_nonlist_unique.head(2)

,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat
2381893,0x0:0x2ab217fc59211262,Sidelines Grille,"Sidelines Grille, 3405 Busbee Dr NW, Kennesaw,...",None,34.034708,-84.575423,restaurant,2.3,3,None,Specific Hours
885585,0x1412bcfc0336fce9:0x33da3c68ef2176ac,Aztlan Foods Corp,"Aztlan Foods Corp, 9110 NW 106th St, Medley, F...",None,25.869479,-80.347611,food products supplier,4.7,8,None,Open


In [11]:
# unimos el df sin las filas con datos duplicados
# y el df con las filas con mayor informacion
data_gmaps_metadata_nonlist_sindups = pd.concat(
    [data_gmaps_metadata_nonlist_unique,
     data_gmaps_metadata_nonlist_dupslen])
data_gmaps_metadata_nonlist_sindups.info()
data_gmaps_metadata_nonlist_sindups.head(2)

<class 'pandas.core.frame.DataFrame'>
Index: 347155 entries, 2381893 to 1080887
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   gmap_id          347155 non-null  object 
 1   name             347155 non-null  object 
 2   address          344916 non-null  object 
 3   description      143863 non-null  object 
 4   latitude         347155 non-null  float64
 5   longitude        347155 non-null  float64
 6   category         347155 non-null  object 
 7   avg_rating       347155 non-null  float64
 8   num_of_reviews   347155 non-null  int64  
 9   price            182074 non-null  object 
 10  hours-state_cat  347155 non-null  object 
dtypes: float64(3), int64(1), object(7)
memory usage: 31.8+ MB


,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat
2381893,0x0:0x2ab217fc59211262,Sidelines Grille,"Sidelines Grille, 3405 Busbee Dr NW, Kennesaw,...",None,34.034708,-84.575423,restaurant,2.3,3,None,Specific Hours
885585,0x1412bcfc0336fce9:0x33da3c68ef2176ac,Aztlan Foods Corp,"Aztlan Foods Corp, 9110 NW 106th St, Medley, F...",None,25.869479,-80.347611,food products supplier,4.7,8,None,Open


## Remover filas duplicadas: Columna 'hours'

In [12]:
data_gmaps_metadata_hours = data_gmaps_metadata[
    column_list_hours].dropna(subset=['hours']).copy()
data_gmaps_metadata_hours['hours_list'] = [list(map(tuple, lst)) for lst in data_gmaps_metadata_hours.hours]
data_gmaps_metadata_hours_list = data_gmaps_metadata_hours.drop(columns=['hours']).copy()

# para que drop_duplicates acepte las columnas anidadas, transformamos las columas en tuple
data_gmaps_metadata_hours_dups = data_gmaps_metadata_hours[
    data_gmaps_metadata_hours_list.assign(
        hours_list = data_gmaps_metadata_hours_list['hours_list'].map(tuple)
        ).duplicated(keep=False) # False muestra todos los duplicados
    ].sort_values(by=['gmap_id']).copy()

# observamos que existen valores duplicados
data_gmaps_metadata_hours_dups.head(4)

,gmap_id,hours,hours_list
768183,0x14589e44e1543481:0x488689a13c898c2d,"[[Sunday, 10:30AM–1:30PM], [Monday, Closed], [...","[(Sunday, 10:30AM–1:30PM), (Monday, Closed), (..."
768183,0x14589e44e1543481:0x488689a13c898c2d,"[[Sunday, 10:30AM–1:30PM], [Monday, Closed], [...","[(Sunday, 10:30AM–1:30PM), (Monday, Closed), (..."
1441346,0x145e9685db62ba25:0xe2c88bc9d79a90b9,"[[Tuesday, 12–9PM], [Wednesday, 12–9PM], [Thur...","[(Tuesday, 12–9PM), (Wednesday, 12–9PM), (Thur..."
1441346,0x145e9685db62ba25:0xe2c88bc9d79a90b9,"[[Tuesday, 12–9PM], [Wednesday, 12–9PM], [Thur...","[(Tuesday, 12–9PM), (Wednesday, 12–9PM), (Thur..."


In [13]:
# para que drop_duplicates acepte las columnas anidadas, transformamos las columas en tuple
data_gmaps_metadata_hours_sindups = data_gmaps_metadata_hours_list.assign(
        hours_list = data_gmaps_metadata_hours_list['hours_list'].map(tuple)
        ).drop_duplicates()
#
data_gmaps_metadata_hours_2join = pd.merge(
    left=data_gmaps_metadata_hours_sindups,
    right=data_gmaps_metadata_hours.drop(
        columns=['hours_list']).reset_index().drop_duplicates(subset='index').set_index('index'),
    how='left').drop(columns=['hours_list'])
# resultado
data_gmaps_metadata_hours_2join.info()
data_gmaps_metadata_hours_2join.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228261 entries, 0 to 228260
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   gmap_id  228261 non-null  object
 1   hours    228261 non-null  object
dtypes: object(2)
memory usage: 3.5+ MB


,gmap_id,hours
0,0x862722f48f2b47ef:0x2d579bd17f79bf9b,"[[Wednesday, 10:30AM–8PM], [Thursday, 10:30AM–..."
1,0x809939273fca9245:0xf2337555fe8f9607,"[[Wednesday, 3–9PM], [Thursday, 3–9PM], [Frida..."
2,0x89b7b7b77cb30111:0xcf3261fc12c26a4,"[[Wednesday, 6AM–8PM], [Thursday, 6AM–8PM], [F..."
3,0x89e75c0234c3372d:0x85de83fd217ede6c,"[[Wednesday, Closed], [Thursday, Closed], [Fri..."


## Remover filas duplicadas: Columna 'misc'

In [14]:
data_gmaps_metadata_misc = data_gmaps_metadata[
    column_list_misc].dropna(subset=['MISC']).copy()
data_gmaps_metadata_misc_dups = data_gmaps_metadata_misc[
    data_gmaps_metadata_misc.assign(
        MISC = data_gmaps_metadata_misc['MISC'].map(tuple)
        ).duplicated(keep=False) # False muestra todos los duplicados
    ].sort_values(by=['gmap_id'])

# observamos que existen valores duplicados
data_gmaps_metadata_misc_dups.head(4)

,gmap_id,MISC
1321055,0x14129c7a5c80b46d:0x33149e690acebdf,{'Accessibility': ['Wheelchair accessible entr...
1321055,0x14129c7a5c80b46d:0x33149e690acebdf,{'Accessibility': ['Wheelchair accessible entr...
768183,0x14589e44e1543481:0x488689a13c898c2d,"{'Accessibility': None, 'Amenities': None, 'At..."
768183,0x14589e44e1543481:0x488689a13c898c2d,"{'Accessibility': None, 'Amenities': None, 'At..."


In [15]:
# para que drop_duplicates acepte las columnas anidadas, transformamos las columas en tuple
data_gmaps_metadata_misc = data_gmaps_metadata[
    column_list_misc].dropna(subset=['MISC']).copy()
data_gmaps_metadata_misc_sindups = data_gmaps_metadata_misc.assign(
    MISC = data_gmaps_metadata_misc['MISC'].map(tuple)
    ).rename(
        columns={'MISC':'misc_tuple'}
        ).drop_duplicates().sort_values(by=['gmap_id'])
data_gmaps_metadata_misc_2join = pd.merge(
    left=data_gmaps_metadata_misc_sindups,
    right=data_gmaps_metadata_misc.reset_index().drop_duplicates(subset='index').set_index('index'),
    how='left').drop(columns=['misc_tuple'])
data_gmaps_metadata_misc_2join.info()
data_gmaps_metadata_misc_2join.head(4)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251139 entries, 0 to 251138
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   gmap_id  251139 non-null  object
 1   MISC     251139 non-null  object
dtypes: object(2)
memory usage: 3.8+ MB


,gmap_id,MISC
0,0x0:0x2ab217fc59211262,{'Accessibility': ['Wheelchair accessible seat...
1,0x14129c7a5c80b46d:0x33149e690acebdf,{'Accessibility': ['Wheelchair accessible entr...
2,0x145898a43de55429:0xe0237e822350dd8e,{'Accessibility': ['Wheelchair accessible entr...
3,0x14589e44e1543481:0x488689a13c898c2d,"{'Accessibility': None, 'Amenities': None, 'At..."


## Resultados

In [16]:
print("casos unicos en nonlist",data_gmaps_metadata_nonlist_sindups['gmap_id'].nunique())
print("casos unicos en hours",data_gmaps_metadata_hours_sindups['gmap_id'].nunique())
print("casos unicos en misc",data_gmaps_metadata_misc_sindups['gmap_id'].nunique())

casos unicos en nonlist 254640
casos unicos en hours 227103
casos unicos en misc 249815


In [17]:
# comparamos los casos en hours con el df de referencia: gmap_id
print(
    "casos que estan nonlist pero no en hours:",
    len(set(data_gmaps_metadata_nonlist_sindups["gmap_id"].unique()).difference(
        set(data_gmaps_metadata_hours_sindups["gmap_id"].unique())))
    )
print(
    "casos que estan hours pero no en nonlist:",
    len(set(data_gmaps_metadata_hours_sindups["gmap_id"].unique()).difference(
        set(data_gmaps_metadata_nonlist_sindups["gmap_id"].unique())))
    )

casos que estan nonlist pero no en hours: 27537
casos que estan hours pero no en nonlist: 0


In [18]:
# comparamos los casos en misc con el df de referencia: gmap_id
print(
    "casos que estan nonlist pero no en misc:",
    len(set(data_gmaps_metadata_nonlist_sindups["gmap_id"].unique()).difference(
        set(data_gmaps_metadata_misc_sindups["gmap_id"].unique())))
    )
print(
    "casos que estan misc pero no en nonlist: ",
    len(set(data_gmaps_metadata_misc_sindups["gmap_id"].unique()).difference(
        set(data_gmaps_metadata_nonlist_sindups["gmap_id"].unique())))
    )

casos que estan nonlist pero no en misc: 4825
casos que estan misc pero no en nonlist:  0


In [19]:
# con base en lo anterior, hacemos un left join donde nonlist es nuestro df de referencia.
data_gmaps_metadata_sindups = pd.merge(pd.merge(
    left=data_gmaps_metadata_nonlist_sindups,
    right=data_gmaps_metadata_hours_2join,
    how='left'),
    right=data_gmaps_metadata_misc_2join,
    how='left')
data_gmaps_metadata_sindups.info()
data_gmaps_metadata_sindups.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351679 entries, 0 to 351678
Data columns (total 13 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   gmap_id          351679 non-null  object 
 1   name             351679 non-null  object 
 2   address          349400 non-null  object 
 3   description      144951 non-null  object 
 4   latitude         351679 non-null  float64
 5   longitude        351679 non-null  float64
 6   category         351679 non-null  object 
 7   avg_rating       351679 non-null  float64
 8   num_of_reviews   351679 non-null  int64  
 9   price            183314 non-null  object 
 10  hours-state_cat  351679 non-null  object 
 11  hours            317726 non-null  object 
 12  MISC             346658 non-null  object 
dtypes: float64(3), int64(1), object(9)
memory usage: 34.9+ MB


,gmap_id,name,address,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours-state_cat,hours,MISC
0,0x0:0x2ab217fc59211262,Sidelines Grille,"Sidelines Grille, 3405 Busbee Dr NW, Kennesaw,...",None,34.034708,-84.575423,restaurant,2.3,3,None,Specific Hours,NaN,{'Accessibility': ['Wheelchair accessible seat...
1,0x1412bcfc0336fce9:0x33da3c68ef2176ac,Aztlan Foods Corp,"Aztlan Foods Corp, 9110 NW 106th St, Medley, F...",None,25.869479,-80.347611,food products supplier,4.7,8,None,Open,"[[Tuesday, 8AM–5:30AM], [Wednesday, 8AM–5:30AM...",NaN


In [20]:
print(
    "casos que estan en df original pero no en df sindups: ",
    len(set(data_gmaps_metadata["gmap_id"].unique()).difference(
        set(data_gmaps_metadata_sindups["gmap_id"].unique())))
    )
print(
    "filas duplicadas removidas (df original - df sindups): ",
    len(data_gmaps_metadata) - len(data_gmaps_metadata_sindups)
    )

casos que estan en df original pero no en df sindups:  0
filas duplicadas removidas (df original - df sindups):  90189
